In [7]:
from datasets import load_dataset

dataset = load_dataset("NTTUNLPTEAM/class-textbook",split='train')
format_func = lambda data: f"text: {data['text']}, metadata: {data['metadata']}, type: {data['type']}, summary: {data['summary']}"

/Users/chihhlinn/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
embeddings = OllamaEmbeddings(model='all-minilm')
text = 'hello world'

query_result = embeddings.embed_query(text)
query_result[:5]

[-0.028497308492660522,
 0.08764791488647461,
 -0.02262119948863983,
 -0.07729446142911911,
 -0.09653183072805405]

In [8]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(format_func(dataset))

# if os.path.exists("./dataset/faiss_db"):
vectorstore = FAISS.from_texts(texts=splits, embedding=OllamaEmbeddings(model='all-minilm'))
# vectorstore.save_local("faiss_index")
    
# else:
# vectorstore = FAISS.load_local(folder_path= "./database/faiss_db",embeddings=OpenAIEmbeddings, allow_dangerous_deserialization=True)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [9]:
from langchain_mistralai import ChatMistralAI
llm = ChatMistralAI(model="mistral-large-latest")

In [10]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join( doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
rag_chain.invoke("give me a summary of the text and generate ten questions")

LocalProtocolError: Illegal header value b'Bearer '